In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Read DF
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_final_adahf.csv'

df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

# LLM Labelling

In [ ]:
#@title Sentiment - LLM Labeling

import plotly.graph_objects as go
import plotly.express as px

# Count the occurrences of each sentiment
sentiment_counts = df['sentiment'].value_counts()

# Define colors for each sentiment
colors = {
    'Negative': px.colors.qualitative.Vivid[9],
    'Neutral': px.colors.qualitative.Pastel[10],
    'Positive': px.colors.qualitative.Vivid[7]
}

# Create the donut chart
fig = go.Figure(data=[go.Pie(labels=sentiment_counts.index,
                             values=sentiment_counts.values,
                             hole=0.7,
                             marker=dict(colors=[colors[sentiment] for sentiment in sentiment_counts.index]))])

# Add total df in the center
fig.add_annotation(text=f"{len(df)} Tweets",
                   x=0.5, y=0.5,
                   font=dict(size=16, color="black"),
                   showarrow=False)

# Add percentage labels
fig.update_traces(textinfo='percent+label', textposition='inside')

# Customize layout
fig.update_layout(title="Tweet Sentiment Distribution - LLM Labeling", title_x=0.5)

# Show the chart
fig.show()

In [ ]:
!pip install -U kaleido
from google.colab import files

fig.write_image("ihsg_llm_sentiment.png", scale=2)
files.download("ihsg_llm_sentiment.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Emotion - LLM Labeling

emotion_order = ['Joy', 'Trust', 'Fear', 'Surprise', 'Sadness', 'Disgust', 'Anger', 'Anticipation']

# Count the occurrences of each emotion
emotion_counts = df['emotion'].value_counts()
emotion_counts = emotion_counts.reindex(emotion_order).dropna()

# Define colors for the donut chart using Plotly's qualitative D3 colors
colors = {
    'Joy': px.colors.qualitative.Set3[11],
    'Trust': px.colors.qualitative.Set3[6],
    'Fear': px.colors.qualitative.Set3[0],
    'Surprise': px.colors.qualitative.Set3[2],
    'Sadness': px.colors.qualitative.Set3[4],
    'Disgust': px.colors.qualitative.Set3[9],
    'Anger': px.colors.qualitative.Set3[3],
    'Anticipation': px.colors.qualitative.Set3[5],
}

# Create the donut chart
fig = go.Figure(data=[go.Pie(labels=emotion_counts.index,
                             values=emotion_counts.values,
                             hole=0.7,
                             marker=dict(colors=[colors[emotion] for emotion in emotion_counts.index]))])



# Add total df in the center
fig.add_annotation(text=f"{len(df)} Tweets",
                   x=0.5, y=0.5,
                   font=dict(size=16, color="black"),
                   showarrow=False)

# Add percentage labels
fig.update_traces(textinfo='percent+label', textposition='inside')

# Customize layout
fig.update_layout(title="Tweet Emotion Distribution - LLM Labeling", title_x=0.5)

# Show the chart
fig.show()


In [ ]:
fig.write_image("ihsg_llm_emotion.png", scale=2)
files.download("ihsg_llm_emotion.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Sentiment over time (3H)
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Convert 'date' column to datetime format
df["date"] = pd.to_datetime(df["created_at"], format="%a %b %d %H:%M:%S %z %Y")

# Round down to the nearest 3-hour interval
df["time_bin"] = df["date"].dt.floor("3H")

# Create custom time bin labels
def format_time_bin(time):
    end_time = time + timedelta(hours=3)
    return f"{time.strftime('%H.%M')} - {end_time.strftime('%H.%M')}"

df["time_bin"] = df["time_bin"].apply(format_time_bin)

# Count occurrences of each sentiment per 3-hour interval
sentiment_counts = df.groupby(["time_bin", "sentiment"]).size().unstack(fill_value=0)

colors = {
    'Negative': px.colors.qualitative.Vivid[9],
    'Neutral': px.colors.qualitative.Pastel[10],
    'Positive': px.colors.qualitative.Vivid[7]
}

# Plotting with Plotly
fig = go.Figure()
for sentiment in sentiment_counts.columns:
    fig.add_trace(go.Scatter(
        x=sentiment_counts.index,
        y=sentiment_counts[sentiment],
        mode='lines+markers',
        name=sentiment,
        line=dict(color=colors.get(sentiment, "black"))
    ))

fig.update_layout(
    title="Sentiment Analysis Over Time (3-hour intervals) on March 18th",
    xaxis_title="Time (3-hour intervals) on March 18th",
    yaxis_title="Count of Sentiments",
    # xaxis=dict(tickangle=-45),
    legend_title="Sentiment",
    template="plotly_white"
)

fig.show()

<ipython-input-5-8de787408f02>:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time_bin"] = df["date"].dt.floor("3H")


In [ ]:
!pip install -U kaleido
from google.colab import files

fig.write_image("ihsg_llm_sentiment_otime3.png", scale=2)
files.download("ihsg_llm_sentiment_otime3.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Emotion over time (1 hour)
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Convert 'date' column to datetime format
df["date"] = pd.to_datetime(df["created_at"], format="%a %b %d %H:%M:%S %z %Y")

# Round down to the nearest 3-hour interval
df["time_bin"] = df["date"].dt.floor("1H")

# Create custom time bin labels
def format_time_bin(time):
    end_time = time + timedelta(hours=1)
    return f"{time.strftime('%H.%M')} - {end_time.strftime('%H.%M')}"

df["time_bin"] = df["time_bin"].apply(format_time_bin)

# Count occurrences of each emotion per 3-hour interval
emotion_counts = df.groupby(["time_bin", "emotion"]).size().unstack(fill_value=0)

colors = {
    'Joy': px.colors.qualitative.Set3[11],
    'Trust': px.colors.qualitative.Set3[6],
    'Fear': px.colors.qualitative.Set3[0],
    'Surprise': px.colors.qualitative.Set3[2],
    'Sadness': px.colors.qualitative.Set3[4],
    'Disgust': px.colors.qualitative.Set3[9],
    'Anger': px.colors.qualitative.Set3[3],
    'Anticipation': px.colors.qualitative.Set3[5],
}

# Plotting with Plotly
fig = go.Figure()
for emotion in emotion_counts.columns:
    fig.add_trace(go.Scatter(
        x=emotion_counts.index,
        y=emotion_counts[emotion],
        mode='lines+markers',
        name=emotion,
        line=dict(color=colors.get(emotion, "black"))
    ))

fig.update_layout(
    title="Emotion Analysis Over Time (1-hour intervals) on March 18th",
    xaxis_title="Time (1-hour intervals) on March 18th",
    yaxis_title="Count of Emotions",
    # xaxis=dict(tickangle=-45),
    legend_title="Emotion",
    template="plotly_white"
)

fig.show()

<ipython-input-3-4bbcdaf99e59>:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time_bin"] = df["date"].dt.floor("1H")


In [ ]:
!pip install -U kaleido
from google.colab import files

fig.write_image("ihsg_llm_emotion_otime1.png", scale=2)
files.download("ihsg_llm_emotion_otime1.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# HuggingFace Labelling

In [ ]:
#@title Sentiment - HuggingFace Labeling

import plotly.graph_objects as go
import plotly.express as px

# Count the occurrences of each sentiment
sentiment_counts = df['sentiment_hf'].value_counts()

# Define colors for each sentiment
colors = {
    'Negative': px.colors.qualitative.Vivid[9],
    'Neutral': px.colors.qualitative.Pastel[10],
    'Positive': px.colors.qualitative.Vivid[7]
}

# Create the donut chart
fig = go.Figure(data=[go.Pie(labels=sentiment_counts.index,
                             values=sentiment_counts.values,
                             hole=0.7,
                             marker=dict(colors=[colors[sentiment] for sentiment in sentiment_counts.index]))])

# Add total df in the center
fig.add_annotation(text=f"{len(df)} Tweets",
                   x=0.5, y=0.5,
                   font=dict(size=16, color="black"),
                   showarrow=False)

# Add percentage labels
fig.update_traces(textinfo='percent+label', textposition='inside')

# Customize layout
fig.update_layout(title="Tweet Sentiment Distribution - HuggingFace Labeling", title_x=0.5)

# Show the chart
fig.show()

In [ ]:
!pip install -U kaleido -q
from google.colab import files

fig.write_image("ihsg_hf_sentiment.png", scale=2)
files.download("ihsg_hf_sentiment.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Emotion - HuggingFace Labeling

emotion_order = ['Joy', 'Trust', 'Fear', 'Surprise', 'Sadness', 'Disgust', 'Anger', 'Anticipation']

# Count the occurrences of each emotion
emotion_counts = df['emotion_hf'].value_counts()
emotion_counts = emotion_counts.reindex(emotion_order).dropna()

# Define colors for the donut chart using Plotly's qualitative D3 colors
colors = {
    'Joy': px.colors.qualitative.Set3[11],
    'Trust': px.colors.qualitative.Set3[6],
    'Fear': px.colors.qualitative.Set3[0],
    'Surprise': px.colors.qualitative.Set3[2],
    'Sadness': px.colors.qualitative.Set3[4],
    'Disgust': px.colors.qualitative.Set3[9],
    'Anger': px.colors.qualitative.Set3[3],
    'Anticipation': px.colors.qualitative.Set3[5],
}

# Create the donut chart
fig = go.Figure(data=[go.Pie(labels=emotion_counts.index,
                             values=emotion_counts.values,
                             hole=0.7,
                             marker=dict(colors=[colors[emotion] for emotion in emotion_counts.index]))])



# Add total df in the center
fig.add_annotation(text=f"{len(df)} Tweets",
                   x=0.5, y=0.5,
                   font=dict(size=16, color="black"),
                   showarrow=False)

# Add percentage labels
fig.update_traces(textinfo='percent+label', textposition='inside')

# Customize layout
fig.update_layout(title="Tweet Emotion Distribution - HuggingFace Labeling", title_x=0.5)

# Show the chart
fig.show()


In [ ]:
!pip install -U kaleido -q
from google.colab import files

fig.write_image("ihsg_hf_emotion.png", scale=2)
files.download("ihsg_hf_emotion.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Tweet Samples

In [ ]:
df_copy = df[((df['sentiment'] == 'Positive') & (df['sentiment_hf'] == 'Negative')) | ((df['sentiment'] == 'Negative') & (df['sentiment_hf'] == 'Positive'))]
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 3 to 2720
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      100 non-null    int64 
 1   created_at               100 non-null    object
 2   favorite_count           100 non-null    int64 
 3   full_text                100 non-null    object
 4   id_str                   100 non-null    int64 
 5   image_url                4 non-null      object
 6   in_reply_to_screen_name  51 non-null     object
 7   lang                     100 non-null    object
 8   location                 50 non-null     object
 9   quote_count              100 non-null    int64 
 10  reply_count              100 non-null    int64 
 11  retweet_count            100 non-null    int64 
 12  tweet_url                100 non-null    object
 13  user_id_str              100 non-null    int64 
 14  username                 100 non-null    objec

In [ ]:
# prompt: create df_copy, where sentiment is positive and sentiment_hf is negative, or the other way around in the df. reset the index in the df_copy,

df_copy_2 = df[((df['sentiment'] == 'Positive') & (df['sentiment_hf'] == 'Negative')) | ((df['sentiment'] == 'Negative') & (df['sentiment_hf'] == 'Positive'))]
df_copy_2 = df_copy.reset_index(drop=True)


In [ ]:
# prompt: print all of the full_text in df-copy and its index in the original df

for index, row in df_copy.iterrows():
  print(f"Index in original df: {df[df['full_text'] == row['full_text']].index.values[0]}")
  print(f"Full Text: {row['full_text']}")
  print("---")


Index in original df: 3
Full Text: But 02 voters stay dumb. Still defending wowo even while getting played like morons. No wonder they're dumb so what do they know about ihsg or rupiah is crashing? 02 voters only understand free lunch
---
Index in original df: 22
Full Text: @ibnux @nuicemedia Bangkeee... Besok mendadak IHSG naik karena udah dihimbau oleh presiden 
---
Index in original df: 32
Full Text: @yudhakhel Terus IHSG jadi naik lagi gitu? Wkwkwkwk
---
Index in original df: 59
Full Text: You get more profit in a week in bitcoin than ihsg for 5 years 
---
Index in original df: 96
Full Text: @nuicemedia Everything except marking up the IHSG
---
Index in original df: 120
Full Text: @ernestprakasa Bisa buzzernya suruh beli saham2 IHSG yg jatuh pake uang THR mereka
---
Index in original df: 125
Full Text: @Tan_Mar3M ITULAH GUNANYA PEJABAT DARI BUZZERP BACOTANNYA TERNAKAN MULYONO PASTI IHSG &amp; EKONOMI JADI MEROKET https://t.co/Eo00XhZbb5
---
Index in original df: 260
Full Text: @tho

# Topic Modelling and Keyword Analysis

In [ ]:
import pandas as pd
from collections import Counter
import re

# Define blacklist of words you want to exclude
blacklist = set([
    'dan', 'yang', 'dengan', 'atau', 'akan', 'di'
    # 'danantara',
    # 'dan', 'dari', 'untuk', 'dengan', 'akan', 'ke', 'di',
    # 'ini', 'itu', 'atau', 'yang', 'a', 'an', 'of', 'to', 'in', 'on', 'at', 'is', 'are'
    # Add more words to blacklist as needed
])

def get_top_ngrams(text, n, top_k=10, blacklist=set()):
    # Preprocess text: lowercase, remove punctuation, split into words
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()

    # Remove blacklisted words for unigram (1-gram) case
    if n == 1:
        words = [word for word in words if word not in blacklist]

    # Create n-grams
    ngrams = zip(*[words[i:] for i in range(n)])
    ngrams = [" ".join(ngram) for ngram in ngrams]

    # For bigrams/trigrams, remove if any component in n-gram is blacklisted
    ngrams = [ngram for ngram in ngrams if all(word not in blacklist for word in ngram.split())]

    # Count n-gram frequencies
    ngram_counts = Counter(ngrams)

    # Get top k n-grams
    top_ngrams = ngram_counts.most_common(top_k)
    return top_ngrams

# Assuming df is your DataFrame and 'llm_summary' contains the text
text = " ".join(df['summary'].astype(str))

# Get top 10 words (unigrams)
top_words = get_top_ngrams(text, 1, top_k=10, blacklist=blacklist)
print("Top 10 words:")
for word, count in top_words:
    print(f"{word}: {count}")

print("\n")

# Get top 10 bigrams
top_bigrams = get_top_ngrams(text, 2, top_k=10, blacklist=blacklist)
# top_bigrams = get_top_ngrams(text, 2, top_k=10)
print("Top 10 bigrams:")
for bigram, count in top_bigrams:
    print(f"{bigram}: {count}")

print("\n")

# Get top 10 trigrams
# top_trigrams = get_top_ngrams(text, 3, top_k=10, blacklist=blacklist)
top_trigrams = get_top_ngrams(text, 3, top_k=10)
print("Top 10 trigrams:")
for trigram, count in top_trigrams:
    print(f"{trigram}: {count}")


Top 10 words:
ihsg: 2489
turun: 514
karena: 453
anjlok: 414
tidak: 330
investor: 183
saham: 168
ekonomi: 167
penurunan: 144
naik: 135


Top 10 bigrams:
ihsg turun: 418
ihsg anjlok: 382
ihsg ihsg: 160
turun karena: 94
anjlok karena: 87
penurunan ihsg: 87
hari ini: 74
ihsg mengalami: 73
mengalami penurunan: 65
ihsg tidak: 64


Top 10 trigrams:
ihsg turun karena: 84
ihsg anjlok karena: 80
ihsg mengalami penurunan: 54
ihsg ihsg anjlok: 39
hari ini ihsg: 34
ihsg ihsg turun: 30
ihsg turun 5: 27
ihsg anjlok ihsg: 25
mengalami penurunan signifikan: 22
ihsg anjlok dan: 21
